In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
dfs = pd.DataFrame({})

for i in range(4,10):
    df = pd.read_csv('./csv/sms-call-internet-mi-2013-11-0{}.csv'.format(i), parse_dates=['time'])
    dfs = dfs.append(df)
df = pd.read_csv('./csv/sms-call-internet-mi-2013-11-10.csv', parse_dates=['time'])
dfs = dfs.append(df)
dfs = dfs.fillna(0)

In [3]:
#Group by hours and aggregate values according to each cell/hour
dfgr = dfs[['cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['time', 'cellid'], as_index=False).sum()
#dfgr['hour'] = dfgr.time.dt.hour+24*(dfgr.time.dt.day-4)
dfgr['hour'] = dfgr.time.dt.hour*60+24*60*(dfgr.time.dt.day-4)+dfgr.time.dt.minute
dfgg = dfgr[['hour', 'cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['hour', 'cellid'], as_index=False).sum()
#dfgg = dfgg.set_index(['hour']).sort_index()
del df
del dfs
print(dfgg)

#Group by hours and aggregate values according to the hour
#dfgr = dfs[['cellid', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['time', 'cellid'], as_index=False).sum()
#dfgr['hour'] = dfgr.time.dt.hour+24*(dfgr.time.dt.day-1)
#dfgg = dfgr[['hour', 'time', 'smsin','smsout', 'callin','callout', 'internet']].groupby(['hour'], as_index=False).sum()
#dfgg = dfgg.set_index(['hour']).sort_index()
#print(dfgg)

           hour  cellid     smsin    smsout    callin   callout   internet
0             0       1  0.134176  0.030875  0.026137  0.055225   9.260190
1             0       2  0.136613  0.029144  0.027356  0.055578   9.282384
2             0       3  0.139208  0.027300  0.028653  0.055954   9.306009
3             0       4  0.127117  0.035890  0.022608  0.054203   9.195904
4             0       5  0.114418  0.033496  0.021727  0.050303   8.312392
...         ...     ...       ...       ...       ...       ...        ...
10079846  10070    9996  0.292904  1.047605  0.178533  0.453114  38.491634
10079847  10070    9997  0.257984  1.034799  0.171990  0.533672  38.994166
10079848  10070    9998  0.257984  1.034919  0.171990  0.520073  38.959929
10079849  10070    9999  0.346751  0.725508  0.171990  0.237047  27.247603
10079850  10070   10000  0.466230  0.439523  0.171990  0.090769  19.402621

[10079851 rows x 7 columns]


In [4]:
l=[] #choosen cells on 20x20 grid
for i in range(4263,6173,100):
    for j in range(i,i+20):
        l.append(j)
#print(l)

In [5]:
df20 = pd.DataFrame({})
for el in dfgg.iterrows():
    if(l.count(int(el[1].cellid))==1):
        tmp = pd.DataFrame([[(el[1].hour), int(el[1].cellid), el[1].smsin, el[1].smsout, el[1].callin, el[1].callout, el[1].internet]], columns=['hour', 'cellid', 'smsin','smsout', 'callin','callout', 'internet'])
        df20=df20.append(tmp)

In [6]:
df20 = df20.set_index(['hour']).sort_index()
print(df20)

         cellid      smsin    smsout    callin   callout    internet
hour                                                                
0.0        4263   3.369988  3.008497  1.019418  2.247168  170.527475
0.0        4264  10.486286  4.615212  0.289121  0.358161  219.670887
0.0        4265   9.008888  4.141032  0.005812  0.000000  219.426285
0.0        4266  16.684253  5.101267  1.477883  1.182153  219.780887
0.0        4267  22.501401  5.969223  2.206053  1.580848  189.152140
...         ...        ...       ...       ...       ...         ...
10070.0    6178   3.697347  1.749578  1.313395  0.782041  186.143305
10070.0    6179   0.825466  2.364973  0.198274  1.052360   56.555300
10070.0    6180   1.395330  2.037542  0.119240  0.744517   34.835548
10070.0    6181   2.357547  1.348504  0.119240  0.119240   31.919190
10070.0    6182   2.472031  1.531638  0.119240  0.119240   29.766796

[403200 rows x 6 columns]


In [7]:
cells = df20.cellid
scaler=MinMaxScaler()
scaled = pd.DataFrame(scaler.fit_transform(df20), columns=df20.columns, index=df20.index)

In [8]:
dfgg=scaled
dfgg['cellid']=cells
print(dfgg)

         cellid     smsin    smsout    callin   callout  internet
hour                                                             
0.0        4263  0.004631  0.005061  0.004469  0.007632  0.053225
0.0        4264  0.014411  0.007763  0.001267  0.001216  0.068620
0.0        4265  0.012381  0.006966  0.000025  0.000000  0.068543
0.0        4266  0.022929  0.008581  0.006479  0.004015  0.068654
0.0        4267  0.030923  0.010041  0.009671  0.005369  0.059060
...         ...       ...       ...       ...       ...       ...
10070.0    6178  0.005081  0.002943  0.005757  0.002656  0.058117
10070.0    6179  0.001134  0.003978  0.000869  0.003574  0.017524
10070.0    6180  0.001918  0.003427  0.000523  0.002529  0.010720
10070.0    6181  0.003240  0.002268  0.000523  0.000405  0.009806
10070.0    6182  0.003397  0.002576  0.000523  0.000405  0.009132

[403200 rows x 6 columns]


In [9]:
dfsi = [] #data from df without indexing
dfsi2 = pd.DataFrame({}) #new df to save
dfso = []
dfso2 = pd.DataFrame({})
dfci = []
dfci2 = pd.DataFrame({})
dfco = []
dfco2 = pd.DataFrame({})
dfin = []
dfin2 = pd.DataFrame({})

dfsi.append(dfgg.loc[0].cellid)
dfso.append(dfgg.loc[0].cellid)
dfci.append(dfgg.loc[0].cellid)
dfco.append(dfgg.loc[0].cellid)
dfin.append(dfgg.loc[0].cellid)

dfsi2["cellid"]=dfsi[0].values
dfso2["cellid"]=dfsi[0].values
dfci2["cellid"]=dfsi[0].values
dfco2["cellid"]=dfsi[0].values
dfin2["cellid"]=dfsi[0].values
j=0
for i in (np.unique(dfgr.hour)): #dfgr.hour.max()+1
    #row[0] - hour, row[1]: df cols
    dfsi.append(dfgg.loc[i].smsin)
    dfsi2["smsin"+str(i)]=dfsi[j+1].values
    
    dfso.append(dfgg.loc[i].smsout)
    dfso2["smsout"+str(i)]=dfso[j+1].values
    
    dfci.append(dfgg.loc[i].callin)
    dfci2["callin"+str(i)]=dfci[j+1].values
    
    dfco.append(dfgg.loc[i].callout)
    dfco2["callout"+str(i)]=dfco[j+1].values
    
    dfin.append(dfgg.loc[i].internet)
    dfin2["internet"+str(i)]=dfin[j+1].values
    j+=1


In [10]:
dfsi2.to_csv("./classif/20x20/classif20x20weekminute_smsin.csv")
dfsi2.head()

,cellid,smsin0,smsin10,smsin20,smsin30,smsin40,smsin50,smsin60,smsin70,smsin80,...,smsin9980,smsin9990,smsin10000,smsin10010,smsin10020,smsin10030,smsin10040,smsin10050,smsin10060,smsin10070
0,4263,0.004631,0.002016,0.006251,0.002847,0.003519,0.000407,0.000407,0.001194,0.001826,...,0.025077,0.019001,0.018682,0.017861,0.010951,0.017750,0.015080,0.014931,0.016405,0.019139
1,4264,0.014411,0.001457,0.001087,0.000000,0.004327,0.003028,0.000000,0.000326,0.001866,...,0.013800,0.014525,0.007580,0.018268,0.013518,0.014460,0.006800,0.007634,0.003549,0.002488
2,4265,0.012381,0.006195,0.001597,0.003211,0.005537,0.002175,0.000016,0.001783,0.001038,...,0.008180,0.009781,0.007411,0.010616,0.020713,0.006005,0.004169,0.006698,0.001038,0.001046
3,4266,0.022929,0.005063,0.003617,0.008055,0.008762,0.004994,0.006907,0.004652,0.000460,...,0.029212,0.015928,0.016417,0.021598,0.019466,0.016889,0.021762,0.010527,0.007257,0.004120
4,4267,0.030923,0.003037,0.004447,0.009231,0.009820,0.006199,0.010464,0.005207,0.000051,...,0.039095,0.019606,0.020425,0.026938,0.018266,0.023911,0.032134,0.012137,0.010875,0.005204


In [11]:
dfso2.to_csv("./classif/20x20/classif20x20weekminute_smsout.csv")
dfso2.head()

,cellid,smsout0,smsout10,smsout20,smsout30,smsout40,smsout50,smsout60,smsout70,smsout80,...,smsout9980,smsout9990,smsout10000,smsout10010,smsout10020,smsout10030,smsout10040,smsout10050,smsout10060,smsout10070
0,4263,0.005061,0.011936,0.003940,0.003443,0.004672,0.002834,0.003927,0.007235,0.003329,...,0.016262,0.012739,0.023016,0.016038,0.011331,0.011233,0.018251,0.020614,0.015612,0.009115
1,4264,0.007763,0.003183,0.004078,0.002045,0.002108,0.000566,0.000973,0.003675,0.001716,...,0.012566,0.010097,0.009936,0.009707,0.004564,0.010787,0.008465,0.003004,0.006569,0.004552
2,4265,0.006966,0.005108,0.005108,0.000914,0.003940,0.001955,0.000010,0.002571,0.001300,...,0.007655,0.003832,0.002165,0.010324,0.002552,0.009025,0.005986,0.003454,0.000000,0.001271
3,4266,0.008581,0.001928,0.007004,0.002794,0.009348,0.001059,0.005903,0.011426,0.002652,...,0.018192,0.006108,0.015455,0.019599,0.018450,0.007787,0.010368,0.013989,0.011830,0.003912
4,4267,0.010041,0.000000,0.008103,0.003218,0.011133,0.000006,0.009154,0.016513,0.003218,...,0.024450,0.007422,0.022325,0.025008,0.027649,0.007997,0.013299,0.019669,0.018626,0.005318


In [12]:
dfci2.to_csv("./classif/20x20/classif20x20weekminute_callin.csv")
dfci2.head()

,cellid,callin0,callin10,callin20,callin30,callin40,callin50,callin60,callin70,callin80,...,callin9980,callin9990,callin10000,callin10010,callin10020,callin10030,callin10040,callin10050,callin10060,callin10070
0,4263,0.004469,0.001906,0.003518,0.003204,0.000000,0.001586,0.001932,0.001906,0.001297,...,0.011328,0.023032,0.027248,0.005427,0.002996,0.008708,0.013708,0.005430,0.007105,0.011112
1,4264,0.001267,0.003383,0.006587,0.003383,0.000000,0.000926,0.005953,0.003383,0.000000,...,0.009157,0.010992,0.026356,0.008468,0.007919,0.008665,0.015463,0.000208,0.012324,0.004017
2,4265,0.000025,0.000025,0.003312,0.000000,0.000025,0.000000,0.008433,0.000000,0.000025,...,0.006675,0.000025,0.012266,0.009001,0.009961,0.014664,0.003312,0.000025,0.006649,0.000000
3,4266,0.006479,0.002237,0.000761,0.003789,0.006362,0.004714,0.005076,0.016726,0.006362,...,0.005987,0.012034,0.023176,0.010865,0.018742,0.012421,0.005735,0.010873,0.003522,0.004710
4,4267,0.009671,0.002945,0.000000,0.005214,0.009671,0.006945,0.002741,0.025841,0.009671,...,0.005496,0.016641,0.030460,0.009679,0.023530,0.009719,0.006930,0.016600,0.002741,0.006930


In [13]:
dfco2.to_csv("./classif/20x20/classif20x20weekminute_callout.csv")
dfco2.head()

,cellid,callout0,callout10,callout20,callout30,callout40,callout50,callout60,callout70,callout80,...,callout9980,callout9990,callout10000,callout10010,callout10020,callout10030,callout10040,callout10050,callout10060,callout10070
0,4263,0.007632,0.001045,0.008385,0.001005,0.001817,0.000020,0.001938,0.001497,0.001506,...,0.017944,0.009316,0.008920,0.015443,0.014687,0.009402,0.005696,0.004291,0.010772,0.003711
1,4264,0.001216,0.003982,0.002621,0.000805,0.070642,0.002796,0.055742,0.006928,0.056377,...,0.020787,0.022615,0.011362,0.027716,0.006384,0.015509,0.006909,0.004468,0.005492,0.003112
2,4265,0.000000,0.005152,0.003948,0.008335,0.120657,0.006953,0.087127,0.005749,0.076494,...,0.020020,0.021909,0.008339,0.020348,0.011361,0.011885,0.010773,0.005769,0.003183,0.003948
3,4266,0.004015,0.003094,0.002249,0.013415,0.051677,0.009207,0.025203,0.007873,0.033180,...,0.021196,0.015891,0.016884,0.020284,0.016907,0.035575,0.017781,0.016727,0.007832,0.005974
4,4267,0.005369,0.002123,0.000000,0.014658,0.022205,0.010871,0.000041,0.010864,0.021489,...,0.022615,0.013354,0.020859,0.021981,0.018370,0.049701,0.022288,0.023725,0.010738,0.005369


In [14]:
dfin2.to_csv("./classif/20x20/classif20x20weekminute_internet.csv")
dfin2.head()

,cellid,internet0,internet10,internet20,internet30,internet40,internet50,internet60,internet70,internet80,...,internet9980,internet9990,internet10000,internet10010,internet10020,internet10030,internet10040,internet10050,internet10060,internet10070
0,4263,0.053225,0.050235,0.056476,0.050453,0.060226,0.053775,0.045518,0.041495,0.040115,...,0.124322,0.094878,0.084977,0.070096,0.073469,0.071066,0.068235,0.069867,0.076565,0.065961
1,4264,0.068620,0.064171,0.055145,0.068394,0.064564,0.067720,0.064246,0.048694,0.046035,...,0.168170,0.088074,0.104093,0.076000,0.083653,0.077177,0.074758,0.074801,0.082112,0.061650
2,4265,0.068543,0.058639,0.053700,0.066220,0.056326,0.046850,0.066279,0.057791,0.046808,...,0.016248,0.023636,0.025123,0.020607,0.022490,0.020476,0.021007,0.019442,0.017748,0.017660
3,4266,0.068654,0.088443,0.058532,0.064681,0.056508,0.053416,0.057534,0.052115,0.045231,...,0.023891,0.024301,0.025682,0.021295,0.023247,0.027752,0.025319,0.022473,0.023498,0.018646
4,4267,0.059060,0.091030,0.052249,0.051210,0.046179,0.048823,0.045237,0.042170,0.038182,...,0.026452,0.025112,0.026010,0.021635,0.024595,0.031946,0.027876,0.023934,0.026605,0.019359
